<a href="https://colab.research.google.com/github/wooohun/BERT-Summarizer/blob/main/BART_Abstractive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# install dependencies
!pip install transformers
!pip install torch
!pip install datasets
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import pandas as pd
import torch
import nltk
import evaluate
from datasets import load_dataset, load_metric, DatasetDict
from evaluate import evaluator

In [ ]:
# install kaggle
!pip install -q kaggle
!mkdir ~/.kaggle

# get kaggle api token from account -> API -> create new API Token
# move kaggle api token to kaggle folder
!cp -v kaggle.json ~/.kaggle

cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
# download dataset
# !chmod 600 /root/.kaggle/kaggl
!kaggle datasets download -d gowrishankarp/newspaper-text-summarization-cnn-dailymail
!unzip newspaper-text-summarization-cnn-dailymail

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
unzip:  cannot find or open newspaper-text-summarization-cnn-dailymail, newspaper-text-summarization-cnn-dailymail.zip or newspaper-text-summarization-cnn-dailymail.ZIP.


In [10]:
# grab datasets
train = load_dataset("cnn_dailymail",'3.0.0', split="train[:51%]")
validation = load_dataset("cnn_dailymail",'3.0.0', split="validation[:51%]")
test = load_dataset("cnn_dailymail",'3.0.0', split="test[:51%]")
# dataset = DatasetDict()
# dataset['train'] = train
# dataset['test'] = test
# dataset['validate'] = validate
train.to_json('train.json')
test.to_json('test.json')
validation.to_json('validation.json')
# formatted as:
# DatasetDict{
#   train: Dataset{ - len = 287113 * x%
#     features:[]
#     num_rows: int 
#   }
#   test: {} - len = 11490
#
# }

data_files = {'train': 'train.json', 'test': 'test.json', 'validation': 'validation.json'}
dataset = load_dataset('json', data_files=data_files)

Creating json from Arrow format:   0%|          | 0/147 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-2acb02846eccc01c/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [23]:
from transformers import BartTokenizerFast, BartForConditionalGeneration

# using fast tokenizer
tokenizer = BartTokenizerFast.from_pretrained('facebook/bart-base')
model  = BartForConditionalGeneration.from_pretrained('../tmp/bart-abst-summarization')

In [24]:
def preprocessing(dataset):
  inputs = [article for article in dataset['article']]
  targets = [target for target in dataset['highlights']]
  tokenized_inputs = tokenizer(inputs, text_target=targets, max_length = 1024, truncation=True)

  # # set tokenizer to encode labels
  # with tokenizer.as_target_tokenizer():
  #   labels = tokenizer(dataset['highlights'], max_length = 128, truncation=True)
  return tokenized_inputs

In [25]:
# process all datasets in batches using fast tokenizer for efficiency
processed_dataset = dataset.map(preprocessing, batched = True)

Map:   0%|          | 0/146428 [00:00<?, ? examples/s]

Map:   0%|          | 0/5860 [00:00<?, ? examples/s]

Map:   0%|          | 0/6818 [00:00<?, ? examples/s]

In [ ]:
tokenizer.batch_decode(processed_dataset['train']['labels'][0], skip_special_tokens=True)

In [26]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

In [ ]:
!pip install --upgrade accelerate

In [ ]:
# create training args
batch_size = 4
logging_steps = len(processed_dataset['train'])

args = Seq2SeqTrainingArguments(
    output_dir = "facebook-bart-base-finetuned-cnn-dailymail",
    learning_rate=5.6e-5,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    predict_with_generate=True,
    fp16=True,
    logging_steps=logging_steps
)

In [20]:
# install metrics
!pip install bert_score
!pip install rouge_score

In [21]:
# load metrics
rouge = evaluate.load('rouge')
bert_score = evaluate.load('bertscore')

In [22]:
import numpy as np

# metric computation function to pass into trainer object
def metric_compute(predicted):
  predictions, labels = predicted
  # decode predictions, labels for metric computation
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  # batch_decode returns list of tokens, use nltk to convert list of tokens to list of sentences
  decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
  decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

  # # compute bert_score
  # bert_score_res = bert_score.compute(
  #     predictions = decoded_preds, references = decoded_labels, use_stemmer=True
  # )
  rouge_res = rouge.compute(
      predictions = decoded_preds, references = decoded_labels, use_stemmer=True
  )
  # result = {key: value.mid.fmeasure * 100 for key, value in bert_score_res.items()}
  # for key, value in rouge_res.items():
  #   result[key] = value.mid.fmeasure * 100
  result = {key: value.mid.fmeasure * 100 for key, value in rouge_res.items()}
  
  return {key: round(val, 4) for key, val in result.items()}

In [27]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [28]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=processed_dataset['train'],
    eval_dataset=processed_dataset['validate'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=metric_compute
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:3                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'args' is not defined

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


In [ ]:
!pip install pynvml nvidia-ml-py3

In [ ]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [ ]:
!pip install accelerate

In [ ]:
from accelerate import Accelerator
from torch.utils.data.dataloader import DataLoader

In [ ]:
training_ds = Dataloader(processed_dataset['train'], batch_size = )

dataloader = DataLoader(processed_dataset, batch_size=args.per_device_train_batch_size)

if args.gradient_checkpointing:
  model.gradient_checkpointing_enable()


accelerator = Accelerator(fp16=args.fp16)
model, optimizer, dataloader = accelerator.prepare(model, adam_bnb_optim, dataloader)

model.train()
for step, batch in enumerate(dataloader, start=1):
  loss = model(**batch).loss
  loss = loss / args.gradient_accumulation_steps
  accelerator.backward(loss)
  if step % args.gradient_accumulation_steps == 0:
    optimizer.step()
    optimizer.zero_grad()


In [ ]:
!git clone https://github.com/huggingface/transformers
!cd transformers
!pip install *

In [ ]:
!pip install -r transformers/examples/pytorch/summarization/requirements.txt

In [ ]:
!pip install git+https://github.com/huggingface/transformers

In [36]:
!python transformers/examples/pytorch/summarization/run_summarization.py \
    --model_name_or_path ../tmp/bart-abst-summarization \
    --do_predict \
    --test_file test.json \
    --test_file test.json \
    --push_to_hub \
    --push_to_hub_model_id finetuned-facebook-bart-base \
    --output_dir bart-abst-summarization \
    --per_device_train_batch_size=4 \
    --per_device_eval_batch_size=4 \
    --gradient_accumulation_steps=4 \
    --overwrite_output_dir \
    --predict_with_generate

2023-05-14 03:16:01.673477: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1543: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case Woohun/finetuned-facebook-bart-base).
  warnings.warn(
05/14/2023 03:16:04 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: False
05/14/2023 03:16:04 - INFO - __main__ - Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
dd

In [12]:
!python transformers/examples/pytorch/summarization/run_summarization.py \
    --model_name_or_path facebook/bart-base \
    --do_train \
    --do_eval \
    --train_file train.json \
    --validation_file validation.json \
    --test_file test.json \
    --output_dir /tmp/bart-abst-summarization \
    --per_device_train_batch_size=4 \
    --per_device_eval_batch_size=4 \
    --gradient_accumulation_steps=4 \
    --overwrite_output_dir \
    --predict_with_generate

2023-05-13 19:49:16.037876: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
05/13/2023 19:49:18 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: False
05/13/2023 19:49:18 - INFO - __main__ - Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': Fals

In [14]:
!cd tmp

/bin/bash: line 0: cd: tmp: No such file or directory


In [16]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid.
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as d

In [17]:
from transformers import BartForConditionalGeneration
model = BartForConditionalGeneration.from_pretrained('../tmp/bart-abst-summarization')

In [18]:
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [35]:
from huggingface_hub import notebook_login
notebook_login()

In [33]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
logging_steps = logging_steps = len(processed_dataset['train'])
batch_size = 4

training_args = Seq2SeqTrainingArguments(
    learning_rate=5.6e-5,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_total_limit=3,
    weight_decay=0.01,
    predict_with_generate=True,
    logging_steps=logging_steps,
    output_dir="bart-abs"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset['train'],
    eval_dataset=processed_dataset['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    # compute_metrics=metric_compute
)